In [1]:
import sys
# Add the directory to the PYTHONPATH
sys.path.append('/home/ayesha.ishaq/Desktop/3DMOTFormer/nuscenes-devkit/python-sdk')

In [2]:
import pickle
import os
import cv2
import json
import operator
from tqdm import tqdm
import argparse
import numpy as np
from PIL import Image
from pathlib import Path
from collections import OrderedDict
from pyquaternion import Quaternion

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.utils.data_classes import Box
from shapely.geometry import box as B
from shapely.geometry import MultiPoint, LineString
from nuscenes.utils.geometry_utils import view_points
from nuscenes.eval.common.loaders import load_prediction
from nuscenes.eval.detection.data_classes import DetectionBox
from nuscenes.eval.common.utils import quaternion_yaw
from nuscenes.eval.tracking.utils import category_to_tracking_name

import torch

from utils.data_util import NuScenesClasses, NuScenesClassesBase, category_to_tracking_name_base

In [3]:
cls_attr_dist = {
    "barrier": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 0,
        "vehicle.parked": 0,
        "vehicle.stopped": 0,
    },
    "bicycle": {
        "cycle.with_rider": 2791,
        "cycle.without_rider": 8946,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 0,
        "vehicle.parked": 0,
        "vehicle.stopped": 0,
    },
    "bus": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 9092,
        "vehicle.parked": 3294,
        "vehicle.stopped": 3881,
    },
    "car": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 114304,
        "vehicle.parked": 330133,
        "vehicle.stopped": 46898,
    },
    "construction_vehicle": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 882,
        "vehicle.parked": 11549,
        "vehicle.stopped": 2102,
    },
    "ignore": {
        "cycle.with_rider": 307,
        "cycle.without_rider": 73,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 165,
        "vehicle.parked": 400,
        "vehicle.stopped": 102,
    },
    "motorcycle": {
        "cycle.with_rider": 4233,
        "cycle.without_rider": 8326,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 0,
        "vehicle.parked": 0,
        "vehicle.stopped": 0,
    },
    "pedestrian": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 157444,
        "pedestrian.sitting_lying_down": 13939,
        "pedestrian.standing": 46530,
        "vehicle.moving": 0,
        "vehicle.parked": 0,
        "vehicle.stopped": 0,
    },
    "traffic_cone": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 0,
        "vehicle.parked": 0,
        "vehicle.stopped": 0,
    },
    "trailer": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 3421,
        "vehicle.parked": 19224,
        "vehicle.stopped": 1895,
    },
    "truck": {
        "cycle.with_rider": 0,
        "cycle.without_rider": 0,
        "pedestrian.moving": 0,
        "pedestrian.sitting_lying_down": 0,
        "pedestrian.standing": 0,
        "vehicle.moving": 21339,
        "vehicle.parked": 55626,
        "vehicle.stopped": 11097,
    },
}

In [4]:

def post_process_coords(corner_coords, imsize=(1600, 900)):
    """
    Get the intersection of the convex hull of the reprojected bbox corners and the image canvas, return None if no
    intersection.
    :param corner_coords: Corner coordinates of reprojected bounding box.
    :param imsize: Size of the image canvas.
    :return: Intersection of the convex hull of the 2D box corners and the image canvas.
    """
    polygon_from_2d_box = MultiPoint(corner_coords).convex_hull
    img_canvas = B(0, 0, imsize[0], imsize[1])

    if polygon_from_2d_box.intersects(img_canvas):
        img_intersection = polygon_from_2d_box.intersection(img_canvas)
        if isinstance(img_intersection, LineString):
          return None
        intersection_coords = np.array([coord for coord in img_intersection.exterior.coords])

        min_x = int(min(intersection_coords[:, 0]))
        min_y = int(min(intersection_coords[:, 1]))
        max_x = int(max(intersection_coords[:, 0]))
        max_y = int(max(intersection_coords[:, 1]))

        return min_x, min_y, max_x, max_y
    else:
        return None

In [5]:
output = "/l/users/ayesha.ishaq/3dmotformer/validation.json"
with open(output, 'r') as f:
    det_2d = json.load(f)

In [6]:
result_file = '/home/ayesha.ishaq/Desktop/3DMOTFormer/detections/val.json'
det_boxes, _ = load_prediction(result_file, 10000, DetectionBox, verbose=True)

Loaded results from /home/ayesha.ishaq/Desktop/3DMOTFormer/detections/val.json. Found detections for 6019 samples.


In [7]:
dataset_dir = "/l/users/ayesha.ishaq/nuScenes"

version_fullname = 'v1.0-trainval'
nusc = NuScenes(version=version_fullname, dataroot=dataset_dir, verbose=True)
sequences_by_name = {scene["name"]: scene for scene in nusc.scene}
splits_to_scene_names = create_splits_scenes()

train_split = 'train' 
val_split = 'val' 
train_scenes = splits_to_scene_names[train_split]
val_scenes = splits_to_scene_names[val_split]
scenes = [train_scenes, val_scenes]

Loading NuScenes tables for version v1.0-trainval...


23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 34.251 seconds.
Reverse indexing ...
Done reverse indexing in 9.2 seconds.


In [8]:
from utils.util import load_clip, extract_clip_feature

clip = load_clip()
labelset = list(NuScenesClasses.keys()) + ['background']
text_features = extract_clip_feature(labelset, clip).detach().cpu().numpy()

Loading CLIP ViT-L/14@336px model...


Finish loading


In [9]:
def base_class_2d(dets):
    for scene in dets:
        for frame in dets[scene]:
            for cam in dets[scene][frame]:
                base_dets = []
                for det in dets[scene][frame][cam]:
                    if det[-1] in NuScenesClasses.values():
                        base_dets.append(det)
                dets[scene][frame][cam] = base_dets
    return dets

                        

In [10]:
import math

def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) of two boxes.
    
    Args:
    box1: A list of 4 numbers representing the coordinates of the first box [x1, y1, x2, y2].
    box2: A list of 4 numbers representing the coordinates of the second box [x1, y1, x2, y2].
    
    Returns:
    iou: IoU value (float) between the two boxes.
    """
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Compute the area of intersection rectangle
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    
    # Compute the area of both the prediction and ground-truth rectangles
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Compute the intersection over union
    iou = inter_area / float(box1_area + box2_area - inter_area)
    
    return iou

def compute_center(box):
    """
    Compute the center of a box.
    
    Args:
    box: A list of 4 numbers representing the coordinates of the box [x1, y1, x2, y2].
    
    Returns:
    center: A tuple (x_center, y_center) representing the center of the box.
    """
    x_center = (box[0] + box[2]) / 2
    y_center = (box[1] + box[3]) / 2
    return (x_center, y_center)

def compute_distance(box1, box2):
    """
    Compute the Euclidean distance between the centers of two boxes.
    
    Args:
    box1: A list of 4 numbers representing the coordinates of the first box [x1, y1, x2, y2].
    box2: A list of 4 numbers representing the coordinates of the second box [x1, y1, x2, y2].
    
    Returns:
    distance: Euclidean distance between the centers of the two boxes.
    """
    center1 = compute_center(box1)
    center2 = compute_center(box2)
    distance = math.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)
    return distance

def find_max_iou_or_closest(projected_box, detection_boxes):
    """
    Compute the IoU between the projected box and all detection boxes, and find the one with the maximum IoU.
    If no overlap, return the detection box closest to the projected box.
    
    Args:
    projected_box: A list of 4 numbers representing the coordinates of the projected box [x1, y1, x2, y2].
    detection_boxes: A list of lists, where each list contains 4 numbers representing the coordinates of a detection box [x1, y1, x2, y2].
    
    Returns:
    best_box: The detection box with the maximum IoU, or the closest box if no overlap.
    best_value: The maximum IoU value or the minimum distance if no overlap.
    """
    if not detection_boxes:  # If there are no detection boxes
        return None

    max_iou = 0
    best_box_class = None

    for det_box in detection_boxes:
        iou = compute_iou(projected_box, det_box[:4])
        if iou > max_iou:
            max_iou = iou
            best_box_class = det_box[-1]

    if max_iou > 0:
        return best_box_class
    else:
        # If no overlap, find the closest box
        min_distance = float('inf')
        closest_box = None

        for det_box in detection_boxes:
            distance = compute_distance(projected_box, det_box[:4])
            if distance < min_distance:
                min_distance = distance
                closest_box = det_box[-1]

        return closest_box


In [11]:
# dets = base_class_2d(det_2d)
nusc_annos = {
            "results": {},
            "meta": None,
        }
nusc_annos["meta"] = {
        "use_camera": False,
        "use_lidar": True,
        "use_radar": False,
        "use_map": False,
        "use_external": False,
    }
classes = list(NuScenesClasses.keys()) + ['barrier']
cameras = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]
for scene_id, scene_name in enumerate(tqdm(val_scenes)):
    scene = sequences_by_name[scene_name]
    first_token = scene['first_sample_token']
    last_token = scene['last_sample_token']
    current_token = first_token
    frame_id = 0
    while True:
        current_sample = nusc.get('sample', current_token)
        frame_ann_tokens = current_sample['anns']
        frame_dets = det_boxes[current_token]
        annos = []
        for det in frame_dets:
            det_dict = det.serialize()
            if det_dict['detection_name'] in NuScenesClasses.keys():
                proj_flag = False
                for cam in cameras:
                    camera = nusc.get('sample_data', current_sample['data'][cam])
                    camera_transform = nusc.get('calibrated_sensor', camera['calibrated_sensor_token'])
                    ego_pose_cam = nusc.get('ego_pose', camera['ego_pose_token'])
                    box = Box(det_dict['translation'],
                              det_dict['size'], 
                              Quaternion(det_dict['rotation']))
                    # Move them to the ego-pose frame.
                    box.translate(-np.array(ego_pose_cam['translation']))
                    box.rotate(Quaternion(ego_pose_cam['rotation']).inverse)

                    # Move them to the calibrated sensor frame.
                    box.translate(-np.array(camera_transform['translation']))
                    box.rotate(Quaternion(camera_transform['rotation']).inverse)

                    # Filter out the corners that are not in front of the calibrated sensor.
                    corners_3d = box.corners()
                    in_front = np.argwhere(corners_3d[2, :] > 0).flatten()
                    corners_3d = corners_3d[:, in_front]

                    # Project 3d box to 2d.
                    corner_coords = view_points(corners_3d, np.array(camera_transform['camera_intrinsic'], dtype=np.float32),
                    True).T[:, :2].tolist()

                    final_coords = post_process_coords(corner_coords)
                    
                    # Skip if the convex hull of the re-projected corners does not intersect the image canvas.
                    if final_coords is None:
                        continue
                    else:
                        class_name = find_max_iou_or_closest(final_coords, det_2d[scene_name][current_token][cam])
                        proj_flag = True
                        break
        
                if not proj_flag or class_name == None:
                    class_name = 7
                nusc_anno = {
                    "sample_token": current_token,
                    "translation": det_dict['translation'] ,
                    "size": det_dict['size'],
                    "rotation": det_dict['rotation'],
                    "velocity": det_dict['velocity'],
                    "detection_name": classes[int(class_name)],
                    "detection_score": det_dict['detection_score'],
                    "attribute_name": max(cls_attr_dist[classes[int(class_name)]].items(), key=operator.itemgetter(1))[0],
                }
                annos.append(nusc_anno)
                print(classes[int(class_name)], det_dict['detection_name'])
            else:
                nusc_anno = {
                    "sample_token": current_token,
                    "translation": det_dict['translation'] ,
                    "size": det_dict['size'],
                    "rotation": det_dict['rotation'],
                    "velocity": det_dict['velocity'],
                    "detection_name": det_dict['detection_name'],
                    "detection_score": det_dict['detection_score'],
                    "attribute_name": det_dict['attribute_name']
                    if det_dict['attribute_name'] is not None
                    else max(cls_attr_dist[det_dict['detection_name']].items(), key=operator.itemgetter(1))[
                        0
                    ],
                }
                annos.append(nusc_anno)
        nusc_annos["results"].update({current_token: annos})  
        
        if current_token == last_token:
            break
        next_token = current_sample['next']
        current_token = next_token
        frame_id += 1


  0%|          | 0/150 [00:00<?, ?it/s]

car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
pedestrian car
car car
car car
car car
pedestrian car
car car
car car
car car
car car
car car
car car
car car
car car
car car
pedestrian car
pedestrian car
barrier car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
pedestrian car
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car truck
pedestrian truck
car truck
car bus
car bus
car trailer
barrier trailer
pedestrian trailer
car trailer
car motorcycle
car motorcycle
car motorcycle
pedestrian bicycle
car motorcycle
car bicycle
car bicycle
car motorcycle
car bicycle
car bicycle
car motorcycle
pedestrian bicycle
car bicycle
car bicycle
car motorcycle
car bicycle
pedestrian bicycle
car motorcycle
car motorcycle
barrier bicycle
car bicycle
pedes

pedestrian pedestrian
pedestrian pedestrian
car pedestrian
car pedestrian
car pedestrian
barrier pedestrian
car pedestrian
car pedestrian
car pedestrian
car pedestrian
car pedestrian
car pedestrian
car pedestrian
barrier pedestrian
car pedestrian
pedestrian pedestrian
car pedestrian
pedestrian pedestrian
car pedestrian
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car car
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car truck
car bus
car motorcycle
car bicycle
car bicycle
barrier bicycle
car bicycle
car motorcycle
pedestrian bicycle
car motorcycle
pedestrian bicycle
car motorcycle
pedestrian motorcycle
pedestrian motorcycle
pedestrian bicycle
pedestrian bicycle
car bicycle
pedestrian 

  1%|          | 1/150 [00:05<13:55,  5.60s/it]

car bicycle
car bicycle
truck bicycle
car motorcycle
barrier pedestrian
pedestrian pedestrian
pedestrian pedestrian
car pedestrian
car pedestrian
car pedestrian
pedestrian pedestrian
car pedestrian
car pedestrian
truck pedestrian
car pedestrian
car pedestrian
car pedestrian
car pedestrian
barrier pedestrian
pedestrian pedestrian
car pedestrian
barrier pedestrian
car pedestrian
barrier pedestrian
truck pedestrian
pedestrian pedestrian
pedestrian pedestrian
car car
truck car
car car
car car
car car
car car
barrier car
car car
car car
car car
car car
pedestrian car
pedestrian car
pedestrian car
car car
car truck
car trailer
barrier bus
pedestrian trailer
car trailer
car bus
car bicycle
car bicycle
pedestrian motorcycle
car bicycle
car bicycle
car motorcycle
pedestrian bicycle
car bicycle
car bicycle
pedestrian bicycle
barrier bicycle
car motorcycle
barrier bicycle
car bicycle
car motorcycle
barrier bicycle
truck bicycle
car bicycle
car motorcycle
barrier bicycle
car bicycle
pedestrian bic

...

  7%|▋         | 11/150 [00:45<09:05,  3.93s/it]

truck pedestrian
truck pedestrian
car pedestrian
pedestrian pedestrian
truck pedestrian
car pedestrian
pedestrian pedestrian
pedestrian pedestrian
pedestrian pedestrian
truck pedestrian
barrier pedestrian
pedestrian pedestrian
car car
barrier car
truck car
barrier car
car car
car car
car car
car car
car car
truck truck
barrier truck
barrier truck
barrier truck
truck bus
barrier bus
barrier bus
car bus
barrier bicycle
barrier bicycle
car motorcycle
barrier bicycle
barrier motorcycle
barrier bicycle
barrier motorcycle
barrier motorcycle
car bicycle
car motorcycle
barrier bicycle
barrier bicycle
barrier bicycle
barrier motorcycle
barrier bicycle
truck bicycle
car motorcycle
barrier bicycle
barrier bicycle
car motorcycle
car bicycle
barrier bicycle
car motorcycle
barrier bicycle
car motorcycle
car motorcycle
car bicycle
car bicycle
car bicycle
barrier motorcycle
barrier bicycle
barrier motorcycle
barrier bicycle
barrier bicycle
barrier bicycle
barrier bicycle
car pedestrian
car pedestrian


  7%|▋         | 11/150 [00:49<10:21,  4.47s/it]

barrier bicycle
pedestrian bicycle
motorcycle motorcycle
car bicycle
motorcycle motorcycle
motorcycle bicycle
motorcycle motorcycle
car motorcycle
car bicycle
car motorcycle
barrier pedestrian
barrier pedestrian
barrier pedestrian
car pedestrian
motorcycle pedestrian
barrier pedestrian
car pedestrian
car pedestrian
barrier pedestrian
motorcycle pedestrian
barrier pedestrian
barrier pedestrian
barrier pedestrian
barrier pedestrian
car pedestrian
car pedestrian
barrier pedestrian
car car
car car
car car
car car
car car
car car
car car
barrier car
truck car
truck car
truck car
barrier car
barrier car
truck truck
truck truck
car truck
truck trailer
truck bus
truck trailer
barrier trailer
motorcycle motorcycle
truck bicycle
truck bicycle
car bicycle
car motorcycle
car bicycle
barrier bicycle
barrier bicycle
barrier bicycle
truck motorcycle
car motorcycle
pedestrian bicycle
truck motorcycle
barrier bicycle
barrier bicycle
barrier bicycle
truck motorcycle
car bicycle
barrier motorcycle
barrie

KeyboardInterrupt: 

In [ ]:
# with open('/home/ayesha.ishaq/Desktop/3DMOTFormer/detections.json', "w") as f:
#     json.dump(nusc_annos, f)

In [ ]:

# # dets = base_class_2d(det_2d)
# cameras = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]
# for scene_id, scene_name in enumerate(tqdm(train_scenes)):
#     scene = sequences_by_name[scene_name]
#     first_token = scene['first_sample_token']
#     last_token = scene['last_sample_token']
#     current_token = first_token
#     frame_id = 0
#     while True:
#         current_sample = nusc.get('sample', current_token)
#         frame_ann_tokens = current_sample['anns']
#         embeddings = []
#         for ann_token in frame_ann_tokens:
#             ann = nusc.get('sample_annotation', ann_token)
#             tracking_name = category_to_tracking_name(ann['category_name'])
#             proj_flag = False
#             if tracking_name is not None:
#                 for cam in cameras:
#                     camera = nusc.get('sample_data', current_sample['data'][cam])
#                     camera_transform = nusc.get('calibrated_sensor', camera['calibrated_sensor_token'])
#                     ego_pose_cam = nusc.get('ego_pose', camera['ego_pose_token'])
#                     box = Box(ann['translation'],
#                               ann['size'], 
#                               Quaternion(ann['rotation']))
#                     # Move them to the ego-pose frame.
#                     box.translate(-np.array(ego_pose_cam['translation']))
#                     box.rotate(Quaternion(ego_pose_cam['rotation']).inverse)

#                     # Move them to the calibrated sensor frame.
#                     box.translate(-np.array(camera_transform['translation']))
#                     box.rotate(Quaternion(camera_transform['rotation']).inverse)

#                     # Filter out the corners that are not in front of the calibrated sensor.
#                     corners_3d = box.corners()
#                     in_front = np.argwhere(corners_3d[2, :] > 0).flatten()
#                     corners_3d = corners_3d[:, in_front]

#                     # Project 3d box to 2d.
#                     corner_coords = view_points(corners_3d, np.array(camera_transform['camera_intrinsic'], dtype=np.float32),
#                     True).T[:, :2].tolist()

#                     final_coords = post_process_coords(corner_coords)
                    
#                     # Skip if the convex hull of the re-projected corners does not intersect the image canvas.
#                     if final_coords is None:
#                         continue
#                     else:
#                         class_name = find_max_iou_or_closest(final_coords, dets[scene_name][current_token][cam])
#                         proj_flag = True
#                         break
            
#                 if tracking_name == 'pedestrian' and 
#                     print('class pred', list(NuScenesClasses.keys())[int(class_name)])
#                     print('orig', tracking_name)

#                 if not proj_flag and tracking_name is not None:
#                     print("missed projection") 
#                     class_name = 7
                
#                 if class_name == None:
#                     print("no 2d dets")
#                     class_name = 7

                
#                 embeddings.append(text_features[int(class_name)])
        
#         if current_token == last_token:
#             break
#         next_token = current_sample['next']
#         current_token = next_token
#         frame_id += 1
    
        

33